# Carregar dados

In [54]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")

qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 1543009



,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo
0,2019,11,Centro-Oeste,0,MS,997.99,0,17,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),1,Masculino
1,2019,11,Centro-Oeste,0,MS,0.00,0,36,212420,Analista de Suporte Computacional,1,Masculino


In [55]:
df_2019 = df[df["ano"] == 2019]
df_2020 = df[df["ano"] == 2020]

print(f"""
Quantidade
df_2019: {len(df_2019)}
df_2020: {len(df_2020)}
""")



Quantidade
df_2019: 162073
df_2020: 222102



# Verificando Dados Gerais

In [56]:
df.describe()

,ano,sigla_uf_id,regiao_id,remuneracao_media,desligamento,idade,ocupacao_id,sexo_id
count,1.543009e+06,1.543009e+06,1.543009e+06,1.543009e+06,1.543009e+06,1.543009e+06,1.543009e+06,1.543009e+06
mean,2.019738e+03,1.882856e+01,2.698468e+00,1.191400e+03,6.938132e-02,3.396295e+01,2.479460e+05,1.206990e+00
std,1.302020e+00,7.084583e+00,1.103642e+00,3.236762e+03,2.541015e-01,9.633748e+00,4.960089e+04,4.051482e-01
min,2.018000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.400000e+01,2.122050e+05,1.000000e+00
25%,2.018000e+03,1.400000e+01,3.000000e+00,0.000000e+00,0.000000e+00,2.700000e+01,2.124050e+05,1.000000e+00
50%,2.020000e+03,2.200000e+01,3.000000e+00,0.000000e+00,0.000000e+00,3.200000e+01,2.124150e+05,1.000000e+00
75%,2.021000e+03,2.500000e+01,3.000000e+00,0.000000e+00,0.000000e+00,3.900000e+01,3.171100e+05,1.000000e+00
max,2.021000e+03,2.600000e+01,4.000000e+00,1.208286e+05,1.000000e+00,9.200000e+01,3.172100e+05,2.000000e+00


# Dados por Estado, Sexo

In [57]:
# graf_sigla_uf = df[["sigla_uf"]].value_counts(ascending=True).plot(kind="barh", 
#                                                                    figsize=(15, 10), 
#                                                                    title="Quantidade de profissionais por UF")
# graf_sigla_uf.get_figure().savefig("./output/gold/graf_sigla_uf.png")

In [58]:
# graf_sexo = df[["sexo"]].value_counts(ascending=True).plot(kind="barh", 
#                                                                    figsize=(10, 8), 
#                                                                    title="Sexo")
# graf_sexo.get_figure().savefig("./output/gold/graf_sexo.png")


# Analisando

In [59]:
# show df grouping by ano, and from this, show the group by sexo
df.groupby(["ano", "sexo"]).size().reset_index(name="qnt")

,ano,sexo,qnt
0,2018,Feminino,101658
1,2018,Masculino,365194
2,2019,Feminino,25888
3,2019,Masculino,136185
4,2020,Feminino,44657
5,2020,Masculino,177445
6,2021,Feminino,147184
7,2021,Masculino,544798


### Salários

Por: ANO 2019, Sexo, Região

In [60]:
# df_ana = df_2019[["sigla_uf", "desligamento", "ocupacao", "sexo", "remuneracao_media"]]
# df_ana = df_2020[["sigla_uf", "desligamento", "ocupacao", "sexo", "regiao", "remuneracao_media"]]
df_ana = df[["sigla_uf", "ano", "desligamento", "ocupacao", "sexo", "regiao", "remuneracao_media"]]
df_ana


,sigla_uf,ano,desligamento,ocupacao,sexo,regiao,remuneracao_media
0,MS,2019,0,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),Masculino,Centro-Oeste,997.99
1,MS,2019,0,Analista de Suporte Computacional,Masculino,Centro-Oeste,0.00
2,MS,2019,0,Programador de Sistemas de Informacao,Masculino,Centro-Oeste,1547.16
3,MS,2019,0,Operador de Computador (Inclusive Microcomputador),Masculino,Centro-Oeste,1456.14
4,MS,2019,0,Operador de Computador (Inclusive Microcomputador),Masculino,Centro-Oeste,1314.90
...,...,...,...,...,...,...,...
1543004,RS,2021,0,Analista de Desenvolvimento de Sistemas,Feminino,Sul,0.00
1543005,RS,2021,0,Analista de Desenvolvimento de Sistemas,Feminino,Sul,0.00
1543006,RS,2021,0,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),Feminino,Sul,0.00
1543007,RS,2021,0,Programador de Sistemas de Informacao,Feminino,Sul,0.00


In [76]:
df_ana["ano"].value_counts()


ano
2021    691982
2018    466852
2020    222102
2019    162073
Name: count, dtype: int64

In [77]:
# df_ana_r remove where remuneracao_media = 0
df_ana_r = df_ana[df_ana["remuneracao_media"] > 0]

In [78]:
df_ana_r["ano"].value_counts()


ano
2020    214433
2019    149064
Name: count, dtype: int64

# Remun. Média

In [62]:
# group df_ana_r by sigla_uf and add a column with mean, max, min, std
df_ana_rm = df_ana_r.groupby(["regiao", "sexo"]).agg(
    {"remuneracao_media": ["mean", "min", "max", "std"]}).copy()

# round values to 2 decimal places
df_ana_rm = df_ana_rm.round(2)

df_ana_rm


remuneracao_media                            
                                    mean     min        max      std
regiao       sexo                                                   
Centro-Oeste Feminino            7500.27  315.69   73241.67  7940.65
             Masculino           8010.56  311.70   85750.97  7655.95
Nordeste     Feminino            3063.45  313.86   43623.47  3248.63
             Masculino           3551.76  311.70   56616.23  3508.21
Norte        Feminino            3683.29  309.67  110716.20  3708.46
             Masculino           3190.75  301.13  105844.80  3665.79
Sudeste      Feminino            4714.59  300.63   70360.39  4344.13
             Masculino           5323.02  300.00  120828.57  4916.12
Sul          Feminino            3470.54  303.33   58268.44  3187.36
             Masculino           4085.85  304.16   84126.00  3375.36

In [63]:
# add a col with difference in percentual between rows of col mean
df_ana_rm["diff"] = df_ana_rm["remuneracao_media"]["mean"].diff() / df_ana_rm["remuneracao_media"]["mean"] * 100

df_ana_rm["diff"] = df_ana_rm["diff"].round(2)

pd.set_option('mode.chained_assignment', None) # remove warning
df_ana_rm["diff"][::2] = 0 

df_ana_rm

remuneracao_media                                diff
                                    mean     min        max      std        
regiao       sexo                                                           
Centro-Oeste Feminino            7500.27  315.69   73241.67  7940.65     NaN
             Masculino           8010.56  311.70   85750.97  7655.95    6.37
Nordeste     Feminino            3063.45  313.86   43623.47  3248.63 -161.49
             Masculino           3551.76  311.70   56616.23  3508.21   13.75
Norte        Feminino            3683.29  309.67  110716.20  3708.46    3.57
             Masculino           3190.75  301.13  105844.80  3665.79  -15.44
Sudeste      Feminino            4714.59  300.63   70360.39  4344.13   32.32
             Masculino           5323.02  300.00  120828.57  4916.12   11.43
Sul          Feminino            3470.54  303.33   58268.44  3187.36  -53.38
             Masculino           4085.85  304.16   84126.00  3375.36   15.06

In [64]:
# export to table in latex to 2 decimal places
df_ana_rm.reset_index().to_latex("./output/silver/df_ana_rm.tex", index=False)

# Demissões

In [65]:
# group df_ana_r by sigla_uf and add a column with mean, max, min, std
df_ana_dm = df_ana_r.groupby(["ano", "regiao", "sexo"]).agg(
    {"desligamento": ["count"]}).copy()

# round values to 2 decimal places
df_ana_dm = df_ana_dm.round(2)

df_ana_dm


desligamento
                                   count
ano  regiao       sexo                  
2019 Centro-Oeste Feminino          3135
                  Masculino        16455
     Nordeste     Feminino          1796
                  Masculino         9765
     Norte        Feminino           801
                  Masculino         1722
     Sudeste      Feminino         14399
                  Masculino        70987
     Sul          Feminino          3470
                  Masculino        26534
2020 Centro-Oeste Feminino          3001
                  Masculino        15757
     Nordeste     Feminino          3939
                  Masculino        17567
     Norte        Feminino          4031
                  Masculino         4085
     Sudeste      Feminino         24390
                  Masculino       103504
     Sul          Feminino          7720
                  Masculino        30439

In [66]:
# group df_ana_r by sigla_uf and add a column with mean, max, min, std
df_ana_dm = df_ana_r.groupby(["regiao", "sexo"]).agg(
    {"desligamento": ["count"]}).copy()

# round values to 2 decimal places
df_ana_dm = df_ana_dm.round(2)

df_ana_dm


desligamento
                              count
regiao       sexo                  
Centro-Oeste Feminino          6136
             Masculino        32212
Nordeste     Feminino          5735
             Masculino        27332
Norte        Feminino          4832
             Masculino         5807
Sudeste      Feminino         38789
             Masculino       174491
Sul          Feminino         11190
             Masculino        56973

In [67]:
# add a col with difference in percentual between rows of col mean
df_ana_dm["diff"] = df_ana_dm["desligamento"]["count"].diff(
) / df_ana_dm["desligamento"]["count"] * 100

df_ana_dm["diff"] = df_ana_dm["diff"].round(2)

pd.set_option('mode.chained_assignment', None)  # remove warning
df_ana_dm["diff"][::2] = 0

df_ana_dm


desligamento     diff
                              count         
regiao       sexo                           
Centro-Oeste Feminino          6136      NaN
             Masculino        32212    80.95
Nordeste     Feminino          5735  -461.67
             Masculino        27332    79.02
Norte        Feminino          4832  -465.65
             Masculino         5807    16.79
Sudeste      Feminino         38789    85.03
             Masculino       174491    77.77
Sul          Feminino         11190 -1459.35
             Masculino        56973    80.36

In [68]:
# export to table in latex to 2 decimal places
df_ana_dm.reset_index().to_latex("./output/silver/df_ana_dm.tex", index=False)


# Cargos

In [69]:
# from df_ana, count the number of ocupacao by sexo
df_ana.groupby(["ocupacao", "sexo"]).size().reset_index()


,ocupacao,sexo,0
0,Administrador de Banco de Dados,Feminino,3058
1,Administrador de Banco de Dados,Masculino,12033
2,Administrador de Redes,Feminino,1985
3,Administrador de Redes,Masculino,16767
4,Administrador de Sistemas Operacionais,Feminino,5190
5,Administrador de Sistemas Operacionais,Masculino,18905
6,Administrador em Segurança da Informação,Feminino,2929
7,Administrador em Segurança da Informação,Masculino,11750
8,Analista de Desenvolvimento de Sistemas,Feminino,108088
9,Analista de Desenvolvimento de Sistemas,Masculino,445439


# Outros

In [70]:
# df_ana[df_ana["sigla_uf"].isin(REGIOES["Norte"])].describe()


In [71]:
# group df_ana by sigla_uf and show mean of valor_remuneracao_media

# df_ana.groupby(["sigla_uf", "sexo"]).mean().reset_index()

# Correlação entre colunas

- Correlação entre Ano, Estado, Idade, Ocupação, Sexo, Cor

In [72]:
# # https://stackoverflow.com/questions/48035381/correlation-among-multiple-categorical-variables-pandas

# df_apenas_ids = df[[
#     "ano", "sigla_uf_id", "idade", "ocupacao_id", "sexo_id", "raca_cor_id"]]

# corr_an = df_apenas_ids.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)
# # corr_an = df_apenas_ids.corr()
# corr_an

Matriz de correlação: The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation; for example, the median house value tends to go up when the median income goes up. When the coefficient is close to –1, it means that there is a strong negative correlation; you can see a small negative correlation between

In [73]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(13, 6))
# sns.heatmap(corr_an, vmax=1, annot=True, linewidths=.5)
# plt.xticks(rotation=30, horizontalalignment="right")
# plt.show()


In [74]:
# from pandas.plotting import scatter_matrix

# scatter_matrix(df_apenas_ids, figsize=(12, 8))

